In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
dataset = pd.read_csv("E:/JupyterProjects/Grocery_Sales_ParticipantsData/Train.csv")

In [3]:
dataset.head()

,Day,GrocerySales
0,1,8418.59
1,2,8447.47
2,3,8336.49
3,4,8579.17
4,5,8524.31


In [4]:
dataset.shape

(692, 2)

In [5]:
dataset.describe()

,Day,GrocerySales
count,692.000000,692.000000
mean,346.500000,8564.733471
std,199.907479,428.815668
min,1.000000,6766.370000
25%,173.750000,8411.536858
50%,346.500000,8677.770000
75%,519.250000,8852.387500
max,692.000000,9290.020000


In [6]:
# Feature Engineering

#Adding mean to dataframe
dataset['Avg90DaySales'] = dataset['GrocerySales'].rolling(90).mean().round(2)
dataset['Avg90DaySales'].fillna(dataset['Avg90DaySales'].iloc[89], inplace=True)

In [7]:
#Adding median to dataframe
dataset['Median90DaySales'] = dataset['GrocerySales'].rolling(90).median().round(2)
dataset['Median90DaySales'].fillna(dataset['Median90DaySales'].iloc[89], inplace=True)

In [8]:
# Set Quarter Number
dataset['QuarterDay'] = np.where(dataset['Day']%90 != 0, dataset['Day']%90, 90)

# Set Monthly Number
dataset['MonthDay'] = np.where(dataset['Day']%30 != 0, dataset['Day']%30, 30)

# Set Weekly Number
dataset['WeekDay'] = np.where(dataset['Day']%7 != 0, dataset['Day']%7, 7)

In [9]:
dataset.iloc[175:195, :]

,Day,GrocerySales,Avg90DaySales,Median90DaySales,QuarterDay,MonthDay,WeekDay
175,176,8864.43,8286.35,8455.57,86,26,1
176,177,8804.02,8288.99,8455.57,87,27,2
177,178,8660.67,8290.03,8455.57,88,28,3
178,179,8862.45,8290.54,8455.57,89,29,4
179,180,8550.54,8286.70,8455.57,90,30,5
180,181,8396.13,8281.75,8443.11,1,1,6
181,182,8510.40,8280.04,8443.11,2,2,7
182,183,8384.20,8274.52,8424.45,3,3,1
183,184,8442.21,8270.10,8424.45,4,4,2
184,185,8242.29,8269.39,8424.45,5,5,3


In [10]:
dataset = dataset.reindex(columns=['Day','WeekDay', 'MonthDay', 'QuarterDay', 'Median90DaySales', 'Avg90DaySales', 'GrocerySales'])

In [11]:
dataset.head()

,Day,WeekDay,MonthDay,QuarterDay,Median90DaySales,Avg90DaySales,GrocerySales
0,1,1,1,1,8527.28,8440.81,8418.59
1,2,2,2,2,8527.28,8440.81,8447.47
2,3,3,3,3,8527.28,8440.81,8336.49
3,4,4,4,4,8527.28,8440.81,8579.17
4,5,5,5,5,8527.28,8440.81,8524.31


In [12]:
dataset = dataset.drop(['Day'], axis=1)

In [13]:
dataset.head()

,WeekDay,MonthDay,QuarterDay,Median90DaySales,Avg90DaySales,GrocerySales
0,1,1,1,8527.28,8440.81,8418.59
1,2,2,2,8527.28,8440.81,8447.47
2,3,3,3,8527.28,8440.81,8336.49
3,4,4,4,8527.28,8440.81,8579.17
4,5,5,5,8527.28,8440.81,8524.31


In [14]:
# Setting X and y variables

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [15]:
# Splitting train and test data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [93]:
# Fitting Random Forest Regression to the dataset

from sklearn.ensemble import GradientBoostingRegressor
regressor = RandomForestRegressor(n_estimators = 300, random_state = 0)
regressor.fit(X_train, y_train)

# Predicting the test set results

y_pred = regressor.predict(X_test)

In [94]:
# Calculating metrics to analyze errors in regression

from sklearn.metrics  import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_test, y_pred))
print(rmse)

215.4434607674461


In [16]:
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics  import mean_squared_error
from math import sqrt

def train_GB(est, depth, lr):
    gb = GradientBoostingRegressor(n_estimators=est, max_depth=depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    print('Est: {} / Depth: {} / Learning Rate: {} /  ---- RMSE: {}'.format(n_est, depth, lr, round(rmse, 3)))

In [18]:
for n_est in [10, 50, 100, 150]:
    for depth in [5, 10, 15, None] :
        for lr in [0.2, 0.5, 0.7]:
            train_GB(n_est, depth, lr)
        print('**********************************************************')
    print('**********************************************************')

Est: 10 / Depth: 5 / Learning Rate: 0.2 /  ---- RMSE: 245.033
Est: 10 / Depth: 5 / Learning Rate: 0.5 /  ---- RMSE: 271.35
Est: 10 / Depth: 5 / Learning Rate: 0.7 /  ---- RMSE: 234.93
**********************************************************
Est: 10 / Depth: 10 / Learning Rate: 0.2 /  ---- RMSE: 246.085
Est: 10 / Depth: 10 / Learning Rate: 0.5 /  ---- RMSE: 241.429
Est: 10 / Depth: 10 / Learning Rate: 0.7 /  ---- RMSE: 243.247
**********************************************************
Est: 10 / Depth: 15 / Learning Rate: 0.2 /  ---- RMSE: 239.069
Est: 10 / Depth: 15 / Learning Rate: 0.5 /  ---- RMSE: 234.753
Est: 10 / Depth: 15 / Learning Rate: 0.7 /  ---- RMSE: 240.338
**********************************************************
Est: 10 / Depth: None / Learning Rate: 0.2 /  ---- RMSE: 240.767
Est: 10 / Depth: None / Learning Rate: 0.5 /  ---- RMSE: 257.071
Est: 10 / Depth: None / Learning Rate: 0.7 /  ---- RMSE: 252.187
**********************************************************
*******

In [116]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics  import mean_squared_error
from math import sqrt

def train_RF(n_est, depth):
    rf = RandomForestRegressor(n_estimators=n_est, max_depth=depth, n_jobs=-1, random_state=0)
    rf_model = rf.fit(X, y)
    y_pred = rf_model.predict(X)
    rmse = sqrt(mean_squared_error(y, y_pred))
    print('Est: {} / Depth: {} ---- Precision: {}'.format(n_est, depth, round(rmse, 3)))

In [122]:
for n_est in [70, 71, 72, 73, 74]:
    for depth in [20, 21, 22, 23, 24, None]:
        train_RF(n_est, depth)

Est: 70 / Depth: 20 ---- Precision: 71.896
Est: 70 / Depth: 21 ---- Precision: 71.883
Est: 70 / Depth: 22 ---- Precision: 71.899
Est: 70 / Depth: 23 ---- Precision: 71.898
Est: 70 / Depth: 24 ---- Precision: 71.9
Est: 70 / Depth: None ---- Precision: 71.904
Est: 71 / Depth: 20 ---- Precision: 71.712
Est: 71 / Depth: 21 ---- Precision: 71.7
Est: 71 / Depth: 22 ---- Precision: 71.716
Est: 71 / Depth: 23 ---- Precision: 71.715
Est: 71 / Depth: 24 ---- Precision: 71.717
Est: 71 / Depth: None ---- Precision: 71.721
Est: 72 / Depth: 20 ---- Precision: 71.932
Est: 72 / Depth: 21 ---- Precision: 71.922
Est: 72 / Depth: 22 ---- Precision: 71.938
Est: 72 / Depth: 23 ---- Precision: 71.936
Est: 72 / Depth: 24 ---- Precision: 71.939
Est: 72 / Depth: None ---- Precision: 71.942
Est: 73 / Depth: 20 ---- Precision: 72.07
Est: 73 / Depth: 21 ---- Precision: 72.058
Est: 73 / Depth: 22 ---- Precision: 72.075
Est: 73 / Depth: 23 ---- Precision: 72.073
Est: 73 / Depth: 24 ---- Precision: 72.076
Est: 73 / 

In [139]:
# Preparing for submission

testSet = pd.read_csv('../Grocery_Sales_ParticipantsData/Test.csv')

In [140]:
# Setting Features for Test Data

# Set Quarter Number
testSet['QuarterDay'] = np.where(testSet['Day']%90 != 0, testSet['Day']%90, 90)

# Set Monthly Number
testSet['MonthDay'] = np.where(testSet['Day']%30 != 0, testSet['Day']%30, 30)

# Set Weekly Number
testSet['WeekDay'] = np.where(testSet['Day']%7 != 0, testSet['Day']%7, 7)

#Adding mean to dataframe
testSet['Avg90DaySales'] = dataset['Avg90DaySales'].iloc[-1]

#Adding median to dataframe
testSet['Median90DaySales'] = dataset['Median90DaySales'].iloc[-1]

#Rearranging columns
testSet = testSet.reindex(columns=['Day','WeekDay', 'MonthDay', 'QuarterDay', 
                                   'Median90DaySales', 'Avg90DaySales'])

# Dropping Day Column
testSet = testSet.drop(['Day'], axis=1)

In [141]:
testSet.head()

,WeekDay,MonthDay,QuarterDay,Median90DaySales,Avg90DaySales
0,7,3,63,8878.6,8886.08
1,1,4,64,8878.6,8886.08
2,2,5,65,8878.6,8886.08
3,3,6,66,8878.6,8886.08
4,4,7,67,8878.6,8886.08


In [142]:
# Fitting Random Forest Regression to the dataset

from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=71, max_depth=21, random_state=0)
regressor.fit(X, y)

# Predicting the test set results

y_pred = regressor.predict(testSet.values)

In [143]:
submission = pd.DataFrame(y_pred)
submission.columns = ['GrocerySales']
submission.to_excel('Submission3.xlsx', index=False)

In [ ]:
# Visualizing the training set results

plt.scatter(X_test, y_test, color='red')
plt.scatter(X_test, y_pred, color='blue')
plt.title('Grocery Sales Forecast (Test Set Results)')
plt.xlabel('Days')
plt.ylabel('Sales')
plt.show()